In [1]:
#!pip install botorch
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

In [2]:
import pandas as pd
#df = pd.read_csv('Case3_2nd_launch_WingsConvCoeffs_Info.csv', sep=";")
df = pd.read_excel('Case5_1st_launch_WingsConvCoeffs_Info.xlsx')
df_conv = df[df['Unnamed: 47'] == 'CONVERGED']
from sklearn.model_selection import train_test_split
hifi, lofi = train_test_split(df_conv, test_size=0.1, random_state=42 )

hifi_x = hifi.alpha0.to_numpy()
lofi_x  = lofi.alpha0.to_numpy()
hifi_y = hifi.Cy0Mean.to_numpy()
lofi_y  = lofi.Cy0Mean.to_numpy()

# normalize features
mean = hifi_x.mean( )
std = hifi_x.std( ) + 1e-6 # prevent dividing by 0
hifi_x = (hifi_x - mean) / std
lofi_x = (lofi_x - mean) / std

# normalize labels
mean, std = hifi_y.mean(), hifi_y.std()
hifi_y = (hifi_y - mean) / std
lofi_y = (lofi_y - mean) / std

# make continguous
#hifi_x, hifi_y = hifi_x.contiguous(), hifi_y.contiguous()
#test_x, test_y = test_x.contiguous(), lofi_y.contiguous()

# Cast them
X_hifi = torch.FloatTensor(hifi_x).unsqueeze(-1)
X_lofi = torch.FloatTensor(lofi_x).unsqueeze(-1)
Y_hifi = torch.FloatTensor(hifi_y).unsqueeze(-1)
Y_lofi = torch.FloatTensor(lofi_y).unsqueeze(-1)


# x_hifi = torch.tensor(df.awa, device=device, dtype=dtype).unsqueeze(-1)  
#x_hifi = torch.tensor(df2['awa'].values).unsqueeze(-1)
#y_hifi = torch.tensor(df2['Cy0Mean'].values).unsqueeze(-1)

In [3]:
from botorch.models import SingleTaskGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood    
from botorch.fit import fit_gpytorch_mll

best_obs_value = lofi_y.max()
bounds = torch.FloatTensor([[-1.9], [1.6]])

In [4]:
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.optim import optimize_acqf

def get_next_points(x_lofi, y_lofi,best_obs_value, bounds, n_points=1):

  single_model = SingleTaskGP(x_lofi, y_lofi)
  mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
  fit_gpytorch_mll(mll)

  EI = qExpectedImprovement( model = single_model, best_f = best_obs_value )

  candidates, _ = optimize_acqf(
    acq_function = EI,
    bounds = bounds,
    q = n_points,
    num_restarts= 200,
    raw_samples = 512 )
  return candidates

In [5]:
get_next_points(X_lofi, Y_lofi, best_obs_value, bounds, n_points=2)

tensor([[-1.7463],
        [-1.9000]])

In [6]:
# Define the Kernel of GP
import gpytorch
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self,X_train, Y_train,likelihood):
        super(ExactGPModel, self).__init__(X_train, Y_train, likelihood)
        # this serve for prior
        #self.mean_module = gpytorch.means.ZeroMean()
        #PeriodicKernel =  gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel())
        #RQKernel =  gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel())
        #self.covar_module = gpytorch.kernels.ProductKernel(PeriodicKernel,RQKernel)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) 

       
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x,covar_x)

In [7]:
## FIT THE MODEL
noise_level = 0.01
noise = noise_level*torch.ones(X_hifi.shape[0])
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=noise, learn_additional_noise=False)

model = ExactGPModel(X_hifi, Y_hifi, likelihood)

# HYPERPARAMETER TUNING
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters())  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
training_iter  = 500
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(X_hifi)
    # Calc loss and backprop gradients
    loss = -mll(output, Y_hifi.squeeze())
    loss.backward()

    print('Iter {}/{} - Loss: {} LenghtParam {} '.format(
        i + 1, training_iter, loss.item(), model.covar_module.base_kernel.lengthscale.detach().numpy() ))
    optimizer.step()

Iter 1/500 - Loss: 4.30378532409668 LenghtParam [[0.6931472]] 
Iter 2/500 - Loss: 4.30088472366333 LenghtParam [[0.6936473]] 
Iter 3/500 - Loss: 4.302562236785889 LenghtParam [[0.6941226]] 
Iter 4/500 - Loss: 4.300481796264648 LenghtParam [[0.69459796]] 
Iter 5/500 - Loss: 4.298645496368408 LenghtParam [[0.6950585]] 
Iter 6/500 - Loss: 4.3156585693359375 LenghtParam [[0.69552064]] 
Iter 7/500 - Loss: 4.298374652862549 LenghtParam [[0.69599724]] 
Iter 8/500 - Loss: 4.305403232574463 LenghtParam [[0.6964353]] 
Iter 9/500 - Loss: 4.294623374938965 LenghtParam [[0.6968891]] 
Iter 10/500 - Loss: 4.306479454040527 LenghtParam [[0.6973277]] 
Iter 11/500 - Loss: 4.301057815551758 LenghtParam [[0.69778603]] 
Iter 12/500 - Loss: 4.317960739135742 LenghtParam [[0.69825727]] 
Iter 13/500 - Loss: 4.314401626586914 LenghtParam [[0.6987272]] 
Iter 14/500 - Loss: 4.3045806884765625 LenghtParam [[0.6991919]] 
Iter 15/500 - Loss: 4.293104648590088 LenghtParam [[0.69967324]] 
Iter 16/500 - Loss: 4.306142

In [29]:
model.eval()
likelihood.eval()

n_runs = 1

for i in range(n_runs):

   print(f"No of optimization: {i}")
   new_candidates = get_next_points(X_lofi, Y_lofi, best_obs_value, bounds, 1)

   with torch.no_grad(), gpytorch.settings.fast_pred_var():
      observed_pred = model(new_candidates[0])
      new_results = torch.FloatTensor([observed_pred.mean.numpy()])

   print(f"New candidates are: {new_candidates}")
   X_lofi = torch.cat([X_lofi, new_candidates ])
   Y_lofi = torch.cat([Y_lofi, new_results])


No of optimization: 0


RuntimeError: Flattening the training labels failed. The most common cause of this error is that the shapes of the prior mean and the training labels are mismatched. The shape of the train targets is torch.Size([1077, 1]), while the reported shape of the mean is torch.Size([1077]).

In [25]:
print(X_lofi.size())
print(torch.cat([X_lofi[0:3], new_candidates ]))
print(new_candidates.size())
with torch.no_grad():
    print(model(new_candidates[0]))

torch.Size([120, 1])
tensor([[-0.4339],
        [ 0.8073],
        [-0.4339],
        [-1.9000]])
torch.Size([1, 1])


RuntimeError: Flattening the training labels failed. The most common cause of this error is that the shapes of the prior mean and the training labels are mismatched. The shape of the train targets is torch.Size([1077, 1]), while the reported shape of the mean is torch.Size([1077]).